# Importing required libraries


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score


# Load the feature-engineered dataset


In [2]:
df = pd.read_csv('../data/feature_engineered_data.csv')

# Define target and features


In [4]:
X = df.drop(columns='FraudResult')
y = df['FraudResult']

# Split the data into train and test sets


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define Models


In [6]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}